In [194]:
!pip install groq

In [195]:
import os

os.environ["GROQ_API_KEY"] = "YOUR GROQ API KEY HERE"

In [196]:
system_prompt = """
    You run in a loop of Thought, Action, PAUSE, Observation.

    At the end of the loop you output an Answer.
    Use Thought to describe your thoughts about the question you have been asked.
    Use Action to run one of the actions available to you - then return PAUSE.
    Observation will be the result of running those actions.

    Your available actions are:

    get_planet_mass:
    e.g. get_planet_mass: "Earth"
    Returns the mass of a specified planet in kilograms.

    calculate:
    e.g. calculate: "4 * 7 / 3"
    Runs a calculation and returns the number - uses Python, so be sure to use floating point syntax if necessary.

    Example session:

    Question: What is the sum of the Earth planet mass and Venus planet mass, all multiplied by 3?

    Thought: I need to first get the masses of Earth and Venus, then calculate their sum multiplied by 3.
    Action: get_planet_mass: "Earth"
    PAUSE

    You will be called again with this:
    Observation: The mass of Earth is 5.97237e24 kg.

    You then continue:

    Thought: Now I need the mass of Venus
    Action: get_planet_mass: "Venus"
    PAUSE

    You will be called again with this:
    Observation: The mass of Venus is 4.8675e24 kg.

    You then continue:

    Thought: I will now calculate the sum of these masses multiplied by 3.
    Action: calculate: "(5.97237e24 + 4.8675e24) * 3"
    PAUSE

    You will be called again with this:
    Observation: The result of the calculation is 3.886941e25.

    You then output:

    Answer: The sum of the Earth and Venus masses, multiplied by 3, is 3.886941e25 kg.
""".strip()

In [197]:
def get_planet_mass(planet_name):
    """
    Returns the mass of a planet in the solar system given its name.

    Parameters:
        planet_name (str): The name of the planet (case-insensitive).

    Returns:
        float: The mass of the planet in kilograms.
        str: Error |message if the planet is not found.
    """
    planet_masses = {
        "mercury": 3.3011e23,  # kg
        "venus": 4.8675e24,    # kg
        "earth": 5.97237e24,   # kg
        "mars": 6.4171e23,     # kg
        "jupiter": 1.8982e27,  # kg
        "saturn": 5.6834e26,   # kg
        "uranus": 8.6810e25,   # kg
        "neptune": 1.02413e26, # kg
    }

    planet_name = planet_name.lower()

    if planet_name in planet_masses:
        return planet_masses[planet_name]
    else:
        return f"Error: '{planet_name}' is not a recognized planet."

def calculate(expression):
    """
    Performs a calculation given a mathematical expression.

    Parameters:
        expression (str): The mathematical expression to evaluate.

    Returns:
        float: The result of the calculation.
        str: Error message if the evaluation fails.
    """
    try:
        return eval(expression)
    except Exception as e:
        return f"Error: {e}"

planet = "Earth"
mass = get_planet_mass(planet)
print(f"The mass of {planet} is {mass} kg.")

result = calculate("10 + 5 * 2")
print(f"The result of the calculation is {result}.")

The mass of Earth is 5.97237e+24 kg.
The result of the calculation is 20.


In [198]:
from groq import Groq

class Agent():
  def __init__(self):
    self.client = Groq(
        api_key=os.environ.get("GROQ_API_KEY")
    )
    self.model_name = "llama3-70b-8192"
    self.system_prompt = system_prompt
    self.messages = []
    if self.system_prompt is not None:
      self.messages.append({"role": "system", "content": self.system_prompt})

  def __call__(self, user_message=""):
    self.messages.append({"role": "user", "content": user_message})
    chat_completion = self.client.chat.completions.create(
        messages=self.messages,
        model=self.model_name,
        temperature=0.5
    )
    self.messages.append({"role": "assistant", "content": chat_completion.choices[0].message.content})
    return chat_completion.choices[0].message.content

In [199]:
agent = Agent()

In [200]:
result = agent("What is the sum of the Earth planet mass and mars planet mass, all multiplied by 2?")

In [201]:
print(result)

Thought: I need to first get the masses of Earth and Mars, then calculate their sum multiplied by 2.

Action: get_planet_mass: "Earth"
PAUSE


In [202]:
observation = get_planet_mass("Earth")
next_prompt = f"Observation: The mass of Earth is {observation} kg."
result = agent(next_prompt)

In [203]:
print(result)

Thought: Now I need the mass of Mars.

Action: get_planet_mass: "Mars"
PAUSE


In [204]:
observation = get_planet_mass("Mars")
next_prompt = f"Observation: The mass of Mars is {observation} kg."
result = agent(next_prompt)

In [205]:
print(result)

Thought: I will now calculate the sum of these masses multiplied by 2.

Action: calculate: "(5.97237e24 + 6.4171e23) * 2"
PAUSE


In [206]:
observation = calculate("(5.97237e24 + 6.4171e23) * 2")
next_prompt = f"Observation: The result of the calculation is {observation} kg."
result = agent(next_prompt)

In [207]:
print(result)

Answer: The sum of the Earth and Mars masses, multiplied by 2, is 1.322816e+25 kg.


In [208]:
agent.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\n\n    At the end of the loop you output an Answer.\n    Use Thought to describe your thoughts about the question you have been asked.\n    Use Action to run one of the actions available to you - then return PAUSE.\n    Observation will be the result of running those actions.\n\n    Your available actions are:\n\n    get_planet_mass:\n    e.g. get_planet_mass: "Earth"\n    Returns the mass of a specified planet in kilograms.\n\n    calculate:\n    e.g. calculate: "4 * 7 / 3"\n    Runs a calculation and returns the number - uses Python, so be sure to use floating point syntax if necessary.\n\n    Example session:\n\n    Question: What is the sum of the Earth planet mass and Venus planet mass, all multiplied by 3?\n\n    Thought: I need to first get the masses of Earth and Venus, then calculate their sum multiplied by 3.\n    Action: get_planet_mass: "Earth"\n    PAUSE\n\n    You will be called ag

In [209]:
import re
from pprint import pprint

def agent_loop(query, max_iterations=10):
  agent = Agent()

  i = 0
  next_prompt = query
  while i < max_iterations:
    i += 1
    result = agent(next_prompt)
    print(result)
    if ("Action" and "PAUSE") in result:
      pattern = r'Action:\s*(\w+):\s*"([^"]+)"'
      match = re.search(pattern, result)
      action_name = match.group(1)
      arguments = match.group(2)

      if action_name == "get_planet_mass":
        observation = get_planet_mass(arguments)
        next_prompt = f"Observation: The mass of {arguments} is {observation} kg."
      elif action_name == "calculate":
        observation = calculate(arguments)
        next_prompt = f"Observation: The result of the calculation is {observation} kg."
      print(next_prompt)
    else:
      break

In [210]:
agent_loop("What is the sum of the Earth planet mass and mars planet mass, all multiplied by 2")

Thought: I need to first get the masses of Earth and Mars, then calculate their sum multiplied by 2.

Action: get_planet_mass: "Earth"
PAUSE
Observation: The mass of Earth is 5.97237e+24 kg.
Thought: Now I need the mass of Mars.

Action: get_planet_mass: "Mars"
PAUSE
Observation: The mass of Mars is 6.4171e+23 kg.
Thought: I will now calculate the sum of these masses multiplied by 2.

Action: calculate: "(5.97237e24 + 6.4171e23) * 2"
PAUSE
Observation: The result of the calculation is 1.322816e+25 kg.
Answer: The sum of the Earth and Mars masses, multiplied by 2, is 1.322816e+25 kg.
